In [1]:
import sys
print(sys.version)
import os
import numpy as np
import pandas as pd
import time

import matplotlib.pyplot as plt
WORKSPACE_PATH  = os.environ['WORKSPACE_PATH']
plt.style.use(WORKSPACE_PATH+'/ROMNet/romnet/extra/postprocessing/presentation.mplstyle')

import cantera as ct
from scipy.integrate import solve_ivp


##########################################################################################
### Input Data

OutputDir          = WORKSPACE_PATH + '/ROMNet/Data_31PSR_Clean/'
FigDir             = OutputDir + '/fig/'

MixtureFile        = 'gri30.yaml'
NRests             = 31
RestVec            = np.logspace(np.log10(1.e-6), np.log10(1.e-4), NRests) # [2.e-5]
#RestVec            = np.concatenate([np.linspace(1.e-6, 1.e-5, 20), np.linspace(2.e-5, 1.e-4, 19)])# [2.e-5]
NPerRest           = 1000

tStratch           = 1.
Nt                 = NPerRest*2

T0Inlet            = 300.
P0Inlet            = ct.one_atm
EqRatioInlet       = 1.

3.8.11 (default, Jun 29 2021, 03:17:05) 
[Clang 12.0.0 (clang-1200.0.32.29)]


In [23]:
##########################################################################################
### Generating Training Data

### Writing Residence Times
FileName = OutputDir+'/orig_data/ResidenceTimes.csv'
np.savetxt(FileName, RestVec)



### Iterating Over Residence Times
DataMat  = None
iStart   = np.zeros(len(RestVec))
iEnd     = np.zeros(len(RestVec))
iSim     = 0
Rest = RestVec[0]
print('Rest = ', Rest)


### Create Mixture
gas   = ct.Solution(MixtureFile)
cpi   = gas.partial_molar_cp/gas.molecular_weights
NSpec = gas.n_species


### Create Inlet
gas.TP      = T0Inlet, P0Inlet 
gas.set_equivalence_ratio(EqRatioInlet, 'CH4:1.0', 'O2:1.0, N2:0.0', basis='mass')
YIn_        = gas.Y
hIn_        = np.dot(gas.X/gas.volume_mole, gas.partial_molar_enthalpies) / gas.density

### Create Combustor
gas.equilibrate('HP')
gas_       = gas
P_         = gas.P
V_         = 1.0
h0         = np.dot(gas.X/gas.volume_mole, gas.partial_molar_enthalpies)/gas.density
gas_.HP    = h0, gas.P
Ny         = gas.n_species+1

Rest_      = Rest
mdot_      = gas.density * V_ / Rest_

y0         = np.array(np.hstack((h0*gas.density*V_, gas.Y*gas.density*V_)), dtype=np.float64)

Rest =  1e-06


6.766612905833977e-14

In [13]:
species = [gas.species_name(i) for i in range(len(gas.Y))]

In [20]:
species[5]

'H2O'